# IBM Coursera Capstone
### Assignment #1 - Creating a Capstone Project Notebook
##### Brett Wiens

This notebook will be used for the various components of the Capstone project. 

1. Write some markdown to explain that this notebook will be mainly used for the capstone project.
2. Import the pandas library as pd.
3. Import the Numpy library as np.
4. Print the following the statement: Hello Capstone Project Course!

In [6]:
import pandas as pd  ## Import the pandas library as pd to be used later
import numpy as np   ## Import the numpy library as np to be used later

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# import k-means from clustering stage
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

In [7]:
print("Hello Capstone Project Course!")    ## A twist on a classic, "Hello, World!"

Hello Capstone Project Course!


### Assignment #2 - Segmenting and Clustering Neighbourhoods in Toronto

In [8]:
import requests

websiteURL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  # Store the website url (rather than inline)
response = requests.get(websiteURL, verify=False)                               # Scrape the website (SSL Verify = False)
html = response.text                                                            # Collect the text from the website

from bs4 import BeautifulSoup                                       # Import the Beautiful Soup Package
soup = BeautifulSoup(html, "html.parser")                           # Parse the HTML from the website
WikiSourceTable = soup.find('table',{'class':'wikitable sortable'}) # Collect the tables from the website

CleanString=""                                 # Initialize a blank string
for tr in WikiSourceTable.find_all('tr'):      # Find all table rows (tr) from the source table
    row1=""                                    # Initialize a blank row (Important because it needs to be appended to)
    for tds in tr.find_all('td'):              # Find all table columns (td) from the source table within the current tr
        row1=row1+","+tds.text                 # Append to the current row the td values with a comma separator
    CleanString=CleanString+row1[1:]           # Append to the CleanString the current row with all columns
    
CleanString = CleanString.replace('\n',',')    # String has new line characters, replace them with commas
CleanList = CleanString.split(',')             # Split the string at the commas (making a list instead)

CleanArray = np.array(CleanList[0:864]).reshape(288,3)     # Not exactly flexible, but takes all but the last list item and converts
                                                           # it into an array of shape 288,3.  The last item is a blank record.
Neighbourhoods = pd.DataFrame(data=CleanArray[0:,0:])      # Converts the array into a pandas dataframe (all rows, all columns)
Neighbourhoods.columns = ['FSA','Borough','Neighbourhood'] # Names the columns appropriately (FSA = Forward Sortation Area)
Neighbourhoods.head(10)                                    # Show the first ten rows for validation

,FSA,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [9]:
## Drop from the dataframe where the Borough column is not assigned by index.  (Do it in place)
Neighbourhoods.drop(Neighbourhoods[Neighbourhoods['Borough'] == 'Not assigned'].index, inplace=True)
## Identify the locations where neighbourhood is not assigned (but as per above, Borough is), and replace with the Borough
Neighbourhoods.loc[Neighbourhoods['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = Neighbourhoods['Borough']

## Identify where FSA and Borough are identical, aggregate other columns (Neighbourhood) with comma separation
F_Neighbourhoods = Neighbourhoods.groupby(['FSA','Borough'], sort=False).agg(', '.join)
F_Neighbourhoods = F_Neighbourhoods.reset_index() # Put it back to the numbered row format with which we are familiar
F_Neighbourhoods.shape # As per the assignment question, demonstrate that the shape of the dataframe is 103,3

(103, 3)

In [10]:
GeoCode = pd.read_csv('http://cocl.us/Geospatial_data',header=None)
GeoCode.columns = ['FSA','Latitude','Longitude']

In [11]:
Neighbourhoods = pd.merge(F_Neighbourhoods, GeoCode, on=['FSA'])
Neighbourhoods.sort_values(by='FSA', ascending = False)
Neighbourhoods.head(12)

,FSA,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938
5,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
6,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
7,M3B,North York,Don Mills North,43.7459058,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7063972,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6571618,-79.3789371


In [12]:
import folium

Toronto_Map = folium.Map(location=[43.653963, -79.387207], zoom_start=10)



for lat, lng, borough, neighbourhood in zip(Neighbourhoods['Latitude'],Neighbourhoods['Longitude'],
                                            Neighbourhoods['Borough'],Neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_Map)

Toronto_Map

In [13]:
CLIENT_ID='NO3ANLL4454R2C354HVUPP1MFEULGJYCHGXQ3DC4YPG1SSYO'
CLIENT_SECRET='BLFEO3JPAWKNBRF3SFOFT4OE5JLDUN440P14IYXMOKYZ0YUF'
VERSION='20190925'
#&ll=40.73,-74.01

LIMIT = 100

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET,         
            lat, lng, 
            VERSION, radius, LIMIT)
        
        results = requests.get(url, verify=False).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                             'Neighbourhood Latitude', 
                             'Neighbourhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [15]:
Neighbourhoods.head()

,FSA,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938


In [16]:
TorontoVenues = getNearbyVenues(
    names=Neighbourhoods['Neighbourhood'],
    latitudes=Neighbourhoods['Latitude'],
    longitudes=Neighbourhoods['Longitude'])

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [17]:
print(TorontoVenues.shape)
print('There are {} uniques categories.'.format(len(TorontoVenues['Venue Category'].unique())))
TorontoVenues.groupby('Neighbourhood').count()

(2264, 7)
There are 278 uniques categories.


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10
...,...,...,...,...,...,...
Willowdale West,5,5,5,5,5,5
Woburn,3,3,3,3,3,3
"Woodbine Gardens, Parkview Hill",11,11,11,11,11,11


In [18]:
# one hot encoding
Toronto_onehot = pd.get_dummies(TorontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = TorontoVenues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.shape

(100, 279)

In [19]:
num_top_venues = 10

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2                  Bar  0.04
3           Steakhouse  0.04
4         Burger Joint  0.03
5           Restaurant  0.03
6                Hotel  0.03
7      Thai Restaurant  0.03
8  American Restaurant  0.03
9       Cosmetics Shop  0.03


----Agincourt----
                             venue  freq
0                   Breakfast Spot   0.2
1               Chinese Restaurant   0.2
2                           Lounge   0.2
3                     Skating Rink   0.2
4                   Sandwich Place   0.2
5                Accessories Store   0.0
6  Molecular Gastronomy Restaurant   0.0
7                           Museum   0.0
8                    Movie Theater   0.0
9                            Motel   0.0


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground  0.33
1                Coffee Shop  0.33
2    

                           venue  freq
0                           Café  0.07
1  Vegetarian / Vegan Restaurant  0.06
2                            Bar  0.05
3             Chinese Restaurant  0.04
4                         Bakery  0.04
5          Vietnamese Restaurant  0.04
6             Mexican Restaurant  0.04
7            Dumpling Restaurant  0.04
8                    Coffee Shop  0.03
9                    Record Shop  0.02


----Christie----
                venue  freq
0                Café  0.20
1       Grocery Store  0.20
2                Park  0.13
3   Convenience Store  0.07
4         Coffee Shop  0.07
5           Nightclub  0.07
6               Diner  0.07
7  Italian Restaurant  0.07
8          Restaurant  0.07
9          Baby Store  0.07


----Church and Wellesley----
                      venue  freq
0               Coffee Shop  0.09
1                   Gay Bar  0.05
2       Japanese Restaurant  0.05
3          Sushi Restaurant  0.04
4                Restaurant  0.03
5        

9  Molecular Gastronomy Restaurant  0.00


----Humber Summit----
                             venue  freq
0                      Pizza Place   0.5
1              Empanada Restaurant   0.5
2                Accessories Store   0.0
3                Mobile Phone Shop   0.0
4                    Movie Theater   0.0
5                            Motel   0.0
6              Monument / Landmark   0.0
7  Molecular Gastronomy Restaurant   0.0
8       Modern European Restaurant   0.0
9               Miscellaneous Shop   0.0


----Humewood-Cedarvale----
                             venue  freq
0                     Hockey Arena  0.25
1                     Tennis Court  0.25
2                            Field  0.25
3                            Trail  0.25
4                Mobile Phone Shop  0.00
5                    Movie Theater  0.00
6                            Motel  0.00
7              Monument / Landmark  0.00
8  Molecular Gastronomy Restaurant  0.00
9       Modern European Restaurant  0.00


--

9       Modern European Restaurant   0.0


----Rouge, Malvern----
                             venue  freq
0             Fast Food Restaurant   1.0
1                Accessories Store   0.0
2                Mobile Phone Shop   0.0
3                           Museum   0.0
4                    Movie Theater   0.0
5                            Motel   0.0
6              Monument / Landmark   0.0
7  Molecular Gastronomy Restaurant   0.0
8       Modern European Restaurant   0.0
9               Miscellaneous Shop   0.0


----Runnymede, Swansea----
                venue  freq
0                Café  0.08
1         Coffee Shop  0.08
2    Sushi Restaurant  0.05
3  Italian Restaurant  0.05
4         Pizza Place  0.05
5       Smoothie Shop  0.03
6   Fish & Chips Shop  0.03
7   French Restaurant  0.03
8              School  0.03
9            Boutique  0.03


----Ryerson, Garden District----
                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.08
2  Mid

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Cosmetics Shop,American Restaurant,Thai Restaurant,Burger Joint,Restaurant,Hotel
1,Agincourt,Sandwich Place,Lounge,Breakfast Spot,Skating Rink,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Coffee Shop,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Coffee Shop,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Construction & Landscaping,Convenience Store
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Dance Studio,Sandwich Place,Pub,Coffee Shop,Pool,Gym,Pharmacy,Electronics Store
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pizza Place,Deli / Bodega,Shopping Mall,Fast Food Restaurant,Bridal Shop,Sandwich Place,Middle Eastern Restaurant,Diner,Supermarket
6,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
7,"Bedford Park, Lawrence Manor East",Pizza Place,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pharmacy,Indian Restaurant,Cupcake Shop,Pub,Restaurant,Café
8,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Café,Steakhouse,Bakery,Farmers Market,Italian Restaurant
9,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Event Space,Ethiopian Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Diner


In [48]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2])

In [49]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [50]:
Toronto_merged = Neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
Toronto_merged = Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
Toronto_merged.drop(Toronto_merged[Toronto_merged['Neighbourhood'] == "Islington Avenue"].index, inplace=True)
Toronto_merged.drop(Toronto_merged[Toronto_merged['Neighbourhood'] == "Newtonbrook, Willowdale"].index, inplace=True)
Toronto_merged.drop(Toronto_merged[Toronto_merged['Neighbourhood'] == "Upper Rouge"].index, inplace=True)
Toronto_merged.tail(60) # check the last columns!

,FSA,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6795571,-79.352188,4.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Pub,Dessert Shop,Bakery
42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.6471768,-79.3815764,4.0,Coffee Shop,Café,Hotel,Restaurant,Gastropub,Bakery,Bar,American Restaurant,Deli / Bodega,Italian Restaurant
43,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.6368472,-79.4281914,4.0,Breakfast Spot,Coffee Shop,Café,Burrito Place,Stadium,Caribbean Restaurant,Restaurant,Bar,Bakery,Intersection
44,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111117,-79.2845772,5.0,Bakery,Bus Line,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Soccer Field,Park,Creperie,Cuban Restaurant
45,M2L,North York,"Silver Hills, York Mills",43.7574902,-79.3747141,3.0,Martial Arts Dojo,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Field
46,M3L,North York,Downsview West,43.7390146,-79.5069436,5.0,Bank,Park,Grocery Store,Hotel,Shopping Mall,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop
47,M4L,East Toronto,"The Beaches West, India Bazaar",43.6689985,-79.3155716,5.0,Park,Sandwich Place,Gym,Burrito Place,Sushi Restaurant,Italian Restaurant,Brewery,Fish & Chips Shop,Ice Cream Shop,Pub
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.6481985,-79.3798169,4.0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Gastropub,Deli / Bodega,Gym
49,M6L,North York,"Downsview, North Park, Upwood Park",43.7137562,-79.4900738,5.0,Basketball Court,Construction & Landscaping,Bakery,Park,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
50,M9L,North York,Humber Summit,43.7563033,-79.5659633,0.0,Pizza Place,Empanada Restaurant,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [51]:
# create map
map_clusters = folium.Map(location=[43.653963, -79.387207], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

IndexError: list index out of range

In [52]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,"Woodbine Gardens, Parkview Hill",43.7063972,-79.309937,0.0,Pizza Place,Fast Food Restaurant,Café,Intersection,Athletics & Sports,Gastropub,Bank,Gym / Fitness Center,Pharmacy,Empanada Restaurant
10,North York,Glencairn,43.709577,-79.4450726,0.0,Pizza Place,Park,Japanese Restaurant,Asian Restaurant,Pub,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
12,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971,0.0,History Museum,Bar,Golf Course,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
17,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.6435152,-79.5772008,0.0,Pet Store,Liquor Store,Shopping Plaza,Coffee Shop,Convenience Store,Beer Store,Pizza Place,Pharmacy,Café,Empanada Restaurant
18,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115,0.0,Pizza Place,Rental Car Location,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Intersection,Cosmetics Shop,Dog Run,Event Space
50,North York,Humber Summit,43.7563033,-79.5659633,0.0,Pizza Place,Empanada Restaurant,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
51,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.2394761,0.0,Intersection,Motel,American Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store
70,Etobicoke,Westmount,43.696319,-79.5322424,0.0,Pizza Place,Intersection,Sandwich Place,Discount Store,Chinese Restaurant,Coffee Shop,Drugstore,Dog Run,Doner Restaurant,Donut Shop
72,North York,Willowdale West,43.7827364,-79.4422593,0.0,Pharmacy,Coffee Shop,Butcher,Discount Store,Pizza Place,Construction & Landscaping,Comfort Food Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
82,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.7816375,-79.3043021,0.0,Pizza Place,Pharmacy,Noodle House,Breakfast Spot,Bank,Fried Chicken Joint,Italian Restaurant,Rental Car Location,Thai Restaurant,Chinese Restaurant


In [53]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.7532586,-79.3296565,1.0,Convenience Store,Park,Food & Drink Shop,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
35,East York,East Toronto,43.685347,-79.3381065,1.0,Park,Convenience Store,Coffee Shop,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
66,North York,York Mills West,43.7527583,-79.4000493,1.0,Bank,Convenience Store,Park,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


In [54]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,Caledonia-Fairbanks,43.6890256,-79.453512,2.0,Park,Women's Store,Fast Food Restaurant,Market,Construction & Landscaping,Dog Run,Festival,Comic Shop,Farmers Market,Falafel Restaurant
40,North York,"CFB Toronto, Downsview East",43.7374732,-79.4647633,2.0,Airport,Park,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
64,York,Weston,43.706876,-79.5181884,2.0,Park,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
83,Central Toronto,"Moore Park, Summerhill East",43.6895743,-79.3831599,2.0,Playground,Gym,Park,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
85,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.8152522,-79.2845772,2.0,Playground,Park,Coffee Shop,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
91,Downtown Toronto,Rosedale,43.6795626,-79.3775294,2.0,Park,Playground,Building,Trail,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [55]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,"Silver Hills, York Mills",43.7574902,-79.3747141,3.0,Martial Arts Dojo,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Field


In [56]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Victoria Village,43.7258823,-79.3155716,4.0,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359,4.0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Gym / Fitness Center
3,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.4647633,4.0,Clothing Store,Coffee Shop,Boutique,Athletics & Sports,Fraternity House,Arts & Crafts Store,Gift Shop,Miscellaneous Shop,Furniture / Home Store,Accessories Store
4,Queen's Park,Queen's Park,43.6623015,-79.3894938,4.0,Coffee Shop,Park,Diner,Gym,Yoga Studio,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Smoothie Shop,Nightclub
7,North York,Don Mills North,43.7459058,-79.352188,4.0,Japanese Restaurant,Caribbean Restaurant,Café,Gym / Fitness Center,Baseball Field,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Downtown Toronto,"Ryerson, Garden District",43.6571618,-79.3789371,4.0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Italian Restaurant,Theater,Fast Food Restaurant,Bakery,Sporting Goods Shop
13,North York,"Flemingdon Park, Don Mills South",43.7258997,-79.340923,4.0,Asian Restaurant,Beer Store,Gym,Coffee Shop,Concert Hall,Bike Shop,Sporting Goods Shop,Sandwich Place,Supermarket,Discount Store
15,Downtown Toronto,St. James Town,43.6514939,-79.3754179,4.0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Cosmetics Shop,Beer Bar,Breakfast Spot,Gastropub,Clothing Store
19,East Toronto,The Beaches,43.6763574,-79.2930312,4.0,Health Food Store,Trail,Neighborhood,Pub,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Diner
20,Downtown Toronto,Berczy Park,43.6447708,-79.3733064,4.0,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Café,Steakhouse,Bakery,Farmers Market,Italian Restaurant


In [57]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East York,Woodbine Heights,43.6953439,-79.3183887,5.0,Skating Rink,Pharmacy,Park,Beer Store,Video Store,Cosmetics Shop,Curling Ice,Eastern European Restaurant,Doner Restaurant,Donut Shop
16,York,Humewood-Cedarvale,43.6937813,-79.4281914,5.0,Trail,Field,Hockey Arena,Tennis Court,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
27,North York,Hillcrest Village,43.8037622,-79.3634517,5.0,Golf Course,Dog Run,Mediterranean Restaurant,Pool,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
29,East York,Thorncliffe Park,43.7053689,-79.3493719,5.0,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Gym,Grocery Store,Housing Development,Intersection,Discount Store,Park
44,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111117,-79.2845772,5.0,Bakery,Bus Line,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Soccer Field,Park,Creperie,Cuban Restaurant
46,North York,Downsview West,43.7390146,-79.5069436,5.0,Bank,Park,Grocery Store,Hotel,Shopping Mall,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop
47,East Toronto,"The Beaches West, India Bazaar",43.6689985,-79.3155716,5.0,Park,Sandwich Place,Gym,Burrito Place,Sushi Restaurant,Italian Restaurant,Brewery,Fish & Chips Shop,Ice Cream Shop,Pub
49,North York,"Downsview, North Park, Upwood Park",43.7137562,-79.4900738,5.0,Basketball Court,Construction & Landscaping,Bakery,Park,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
56,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.6911158,-79.4760133,5.0,Sandwich Place,Convenience Store,Restaurant,Check Cashing Service,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
60,North York,Downsview Northwest,43.7616313,-79.5209994,5.0,Discount Store,Athletics & Sports,Gym / Fitness Center,Grocery Store,Liquor Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [58]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534,6.0,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant


In [59]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 7, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,Scarborough Village,43.7447342,-79.2394761,7.0,Playground,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Field


In [60]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 8, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,Downsview Central,43.7284964,-79.4956974,8.0,Food Truck,Baseball Field,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
57,North York,"Emery, Humberlea",43.7247659,-79.5322424,8.0,Paper / Office Supplies Store,Baseball Field,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [61]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 9, Toronto_merged.columns[[1] + list(range(2, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.6509432,-79.5547244,9.0,Bank,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store
